### 경사하강법 이용행렬분해 예제 
행렬분해과정
- 1) 𝑃, 𝑄 를임의의값들을가진행렬로초기화한다
- 2) 𝑃, 𝑄𝑇를행렬곱하여 መ 𝑅을생성한다
- 3) 𝑅과 መ 𝑅을빼손실을구한다
     - 이때손실은𝑅의값이존재하는부분들과 መ 𝑅예측값들간의차이를의미한다
- 4) 𝑃, 𝑄𝑇를손실이최소화하는방향으로업데이트하며, 수렴할때까지작업을반복한다

1) 원본행렬 생성

In [2]:
import numpy as np

# 원본 사용자 - 아이템 행렬 생성
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
             [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape
K=3 # 잠재요인의 개수(차원)

In [3]:
# P, Q를 임의의 값들을 가진 행렬로 초기화

np.random.seed(1)
P = np.random.normal(scale = 1./K, size = (num_users, K))
Q = np.random.normal(scale = 1./K, size = (num_items, K))

In [7]:
P

array([[ 0.54144845, -0.2039188 , -0.17605725],
       [-0.35765621,  0.28846921, -0.76717957],
       [ 0.58160392, -0.25373563,  0.10634637],
       [-0.08312346,  0.48736931, -0.68671357]])

In [8]:
Q

array([[-0.1074724 , -0.12801812,  0.37792315],
       [-0.36663042, -0.05747607, -0.29261947],
       [ 0.01407125,  0.19427174, -0.36687306],
       [ 0.38157457,  0.30053024,  0.16749811],
       [ 0.30028532, -0.22790929, -0.04096341]])

RMSE 측정 함수 생성

In [19]:
# 손실은 𝑅의 값이 존재하는 부분들과 መ 𝑅예측값들 간의 차이를 의미한다

# 실제 행렬과 예측행렬 오차를 구하는 함수 생성
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    # 2) 예측행렬 생성 - 𝑃, 𝑄𝑇를 행렬곱해서 𝑅을 생성.
    full_pred_matrix = np.dot(P, Q.T)
    
    # R에서 NaN이 아닌 값의 위치 인덱스를 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    # NaN이 아닌 값들만을 R과 full_pred_matrix에서 추출
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse
    

경사하강법이용 행렬분해실시

In [20]:
# 원본행렬에서 결측값이 아닌 값의 행, 열 , 실제 값을 튜플형태로 리스트안에 담기.
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] >0]

steps = 1000 #훈련횟수
learning_rate = 0.01
r_lambda = 0.01

for step in range(steps):
    for i, j, r in non_zeros:
        eij = r - np.dot(P[i, :], Q[j,:].T) # i번째 행, j번째 열 값의 오차.
        
        # L2정규화를 적용한 SGD기법 이용 P,Q 업데이트
        P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda * P[i,:])
        Q[i,:] = Q[i,:] + learning_rate * (eij * P[i,:] - r_lambda * Q[j,:])
    
    # 손실을 측정
    rmse = get_rmse(R, P, Q, non_zeros) # 매 스탭이 끝날 때마다 rmse값 산출
    
    if(step% 50) ==0:
        print('### iteration step: {} rmse: {}'.format(step, rmse))

### iteration step: 0 rmse: 3.1226933820510543
### iteration step: 50 rmse: 1.8397551332522026
### iteration step: 100 rmse: 1.8048182903673562
### iteration step: 150 rmse: 1.8640580934405333
### iteration step: 200 rmse: 1.851766596300674
### iteration step: 250 rmse: 1.839023233290267
### iteration step: 300 rmse: 1.80221122833616
### iteration step: 350 rmse: 1.8863071206396325
### iteration step: 400 rmse: 1.8901497220592365
### iteration step: 450 rmse: 1.8851292025431967
### iteration step: 500 rmse: 1.882269035983131
### iteration step: 550 rmse: 1.8907475767807382
### iteration step: 600 rmse: 1.9110896706576719
### iteration step: 650 rmse: 1.9303188742412007
### iteration step: 700 rmse: 1.9245008959012655
### iteration step: 750 rmse: 1.7783969856839055
### iteration step: 800 rmse: 1.9453626158466213
### iteration step: 850 rmse: 1.925929668511872
### iteration step: 900 rmse: 1.939452211702705
### iteration step: 950 rmse: 1.9774265086410294


In [21]:
# 경사하강법이용 행렬분해예측 결과

pred_matrix = np.dot(P, Q.T)
print('예측행렬: \n', np.round(pred_matrix, 3))

예측행렬: 
 [[  3.969 -17.248  16.918   2.011   0.494]
 [ -0.423   4.97   -1.879   3.045   0.069]
 [  0.884   0.224   3.106   4.228   0.264]
 [  0.313  -0.194  -1.464  -0.847   0.066]]


In [22]:
# 원본행렬 출력 결과
print(R)

# SGD 기법을 이용한 행렬분해를 통해 - > 손실을 최소화 하는 방향으로 학습

[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


## 콘텐츠기반필터링실습

In [30]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv("/content/tmdb_5000_movies.csv")
movies.head()

FileNotFoundError: [Errno 2] File b'/content/tmdb_5000_movies.csv' does not exist: b'/content/tmdb_5000_movies.csv'